In [68]:
import os

# Add OpenAI library
import openai

# Get Configuration Settings
from dotenv import load_dotenv
load_dotenv()

True

In [69]:
# Configure OpenAI API using Azure OpenAI
openai.api_key = os.getenv("API_KEY")
openai.api_base = os.getenv("ENDPOINT")
openai.api_type = "azure"  # Necessary for using the OpenAI library with Azure OpenAI
openai.api_version = os.getenv("OPENAI_API_VERSION")  # Latest / target version of the API

In [70]:
from langchain.embeddings import OpenAIEmbeddings

In [71]:
# OpenAI Settings
model_deployment = "text-embedding-ada-002"
# SDK calls this "engine", but naming it "deployment_name" for clarity

model_name = "text-embedding-ada-002"

In [72]:
openai_embeddings: OpenAIEmbeddings = OpenAIEmbeddings(
    openai_api_version = os.getenv("OPENAI_API_VERSION"), openai_api_key = os.getenv("API_KEY"),
    openai_api_base = os.getenv("ENDPOINT"), openai_api_type = "azure"
)

### Sample with Junior Lessons

In [73]:
from langchain_chroma import Chroma

In [74]:
# Junior
vector_store_junior = Chroma(
    collection_name="Junior_Lessons",
    embedding_function=openai_embeddings,
    persist_directory="/Users/mac/Documents/Gospel-Companion/chroma_afc_sunday_school_lessons_db",  # Where to save data locally, remove if not neccesary
)

In [75]:
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory, ConversationBufferWindowMemory
from langchain import PromptTemplate

In [76]:
template_j = """
Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Junior Sunday school lessons".
------
<ctx>
{context}
</ctx>
------
<hs>
{history}
</hs>
------
{question}
Answer:
"""
prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template_j,
)

In [77]:
# print(prompt.format(history="The lesson was about the story of the Good Samaritan.", context="The Junior Sunday school lessons", question="What is the story of the Good Samaritan?"))

In [78]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

In [79]:
import langchain
langchain.verbose = True

In [80]:
llm = ChatOpenAI(temperature = 0.6, openai_api_key = os.getenv("API_KEY"), openai_api_base = os.getenv("ENDPOINT"), model_name="gpt-35-turbo", engine="Voicetask")

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [195]:
def context_document_retreival_similarity(question_summary):
    results = vector_store_junior.similarity_search(question_summary, k=5)
    context = ""
    sources = []
    for result in results:
        context += result.page_content + "\n"
        new_source = result.metadata['source'].split("/")[-1]
        if new_source not in sources:
            sources.append(new_source)
    return context, sources

In [196]:
print(context_document_retreival_similarity("What are the steps to creation? Answer in bullet form using the days and the creation")[1])

['Lesson1_Junior.txt', 'Lesson92_Junior.txt']


### Conversation history summary

In [238]:
from langchain.prompts import PromptTemplate
def conversation_history_prompt(history, question):
    # Define the template string for summarizing conversation history
    template_summary = """
    "Given a chat history (delimited by <hs></hs>) and the latest user question \
    which might reference context in the chat history, formulate a standalone question \
    which can be understood without the chat history. Do NOT answer the question, \
    just reformulate it if needed and otherwise return it as is.
    ------
    <hs>
    {history}
    </hs>
    ------
    Question: {question}
    Summary:
    """

    # Create a PromptTemplate object
    prompt = PromptTemplate(
        input_variables=["history", "question"],
        template=template_summary,
    )

    return prompt.format(history=history, question=question)

In [239]:
def get_conversation_summary(history, question):
    # Get the conversation summary prompt
    formatted_prompt = conversation_history_prompt(history, question)

    # Query the Azure OpenAI LLM with the formatted prompt
    response = openai.ChatCompletion.create(
        engine="Voicetask",  # Replace with your Azure OpenAI deployment name
        # prompt=formatted_prompt,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": formatted_prompt}
        ],
        # max_tokens=50,
        temperature=0.5
    )
    
    # Extract and return the summary from the response
    return response.choices[0].message['content']

In [240]:
history = """
Human: What are the steps to creation? Answer in bullet form using the days and the creation
AI: - Day 1: God created light
- Day 2: God divided the waters and created the sky
- Day 3: God created dry land and vegetation
- Day 4: God created the sun, moon, and stars
- Day 5: God created sea creatures and birds
- Day 6: God created land animals and humans
- Day 7: God rested
Human: What is the proof that God actually rested on the 7th day of creation
AI: The proof that God rested on the seventh day of creation is found in Genesis 2:2-3 in which it is stated that "And on the seventh day God ended his work which he had made; and he rested on the seventh day from all his work which he had made. And God blessed the seventh day, and sanctified it: because that in it he had rested from all his work which God created and made."
Human: What really is then the reason Christians worship on Sunday? And is this correct according to the bible as soon groups still say that the Sabbath day, as in the day for worship, Saturday.
AI: According to the Junior Sunday School lessons, Christians worship on Sunday because it is the day on which Jesus rose from the dead, the day on which he made His appearances to the disciples, the day on which the Holy Ghost was given, and the day on which the disciples met to break bread in commemoration of the Lord's Supper and brought their offerings to the Lord. It has been observed by all true Christians from that time down to the present day. The Bible also mentions keeping the Sabbath day holy, which is the seventh day of the week, but it is important to note that the Sabbath day was given specifically to the Children of Israel, and the first day of the week, Sunday, became accepted as the day of worship immediately after Jesus' resurrection. Ultimately, the decision of which day to worship on is a matter of interpretation and personal belief.
Human: What are some of the most controversial and debated doctrines of the Bible?
AI: Some of the most controversial and debated doctrines of the Bible include the interpretation of the creation story, the nature of God (such as the Trinity), the concept of salvation and grace, the role of women in the church, and the interpretation of prophecy and end times.
Human: What about eternal Salvation, when one believes that once he is born again, he can continue in sin but he still remians born again.
AI: According to the Junior Sunday School lessons, being saved at one time does not necessarily mean that a person will always be saved. Unless a person lives for Jesus and does not permit sin to enter their life, they will lose their right to eternal life. The scripture in Ezekiel 18:26 states, "When a righteous man turneth away from his righteousness, and committeth iniquity, and dieth in them; for his iniquity that he hath done shall he die." In order to be ready to go to Heaven, old things must pass away and the sinful nature must be purged, and all things must become new in our lives. Therefore, it is not possible for a person to continue in sin and still remain born again.
Human: What can you tell me about the book of Hebrews that a new Christian should know?
AI: The book of Hebrews was written by the Apostle Paul to the Hebrew Christians, those who had believed on Jesus. They had been brought up to obey the Law of Moses, and they knew all about the sacrifices that should be made. But those sacrifices had been given as pictures to show the people in Old Testament times that Jesus would some day come to shed His Blood to redeem mankind from sin. When Jesus came, He did just as the sacrifices had pictured: He was crucified on Calvary, shedding His Blood. After that there was no need to sacrifice any more animals. Additionally, the book of Hebrews provides instructions on how to be prepared to meet Jesus and encourage Christians to hold fast until Jesus comes.
"""

In [241]:
new_question = get_conversation_summary(history, "What sacrifices did they make?")

In [242]:
new_question

"What sacrifices did they make according to the chat history's reference to the Law of Moses and the sacrifices mentioned in the book of Hebrews?"

In [243]:
documents, sources = context_document_retreival_similarity(new_question)

In [244]:
# print(prompt.format(history=history, question=new_question, context=documents))
full_prompt = prompt.format(history=history, question=new_question, context=documents)

In [245]:
print(full_prompt)


Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Junior Sunday school lessons".
------
<ctx>
Bible Text: [HEB:1:1-14].

Lesson 432 - Junior

Memory Verse: [HEB:1:1-14].

Notes"Go ye therefore, and teach all nations, baptizing them in the name of the Father, and of the Son, and of the Holy  Ghost" (Matthew 28:19).

The Sacrifices Fulfilled

The Book of Hebrews was written by the Apostle Paul to the Hebrew Christians, those who had believed on Jesus. They had been brought up to obey the Law of Moses, and they knew all about the sacrifices that should be made. But those sacrifices had been given as pictures to show the people in Old Testament times that Jesus would some day come to shed H

In [246]:
def qa_response(prompt):

    # Query the Azure OpenAI LLM with the formatted prompt
    response = openai.ChatCompletion.create(
        engine="Voicetask",  # Replace with your Azure OpenAI deployment name
        # prompt=formatted_prompt,
        messages=[
            {"role": "system", "content": "You are a helpful assistant vast in the Bible and its Doctrines."},
            {"role": "user", "content": prompt}
        ],
        # max_tokens=50,
        temperature=0.5
    )
    
    # Extract and return the summary from the response
    return response.choices[0].message['content']

In [247]:
print(qa_response(full_prompt))

According to the chat history's reference to the Law of Moses and the sacrifices mentioned in the book of Hebrews, the Israelites were commanded to bring animals to the Tabernacle to be killed as a sacrifice to God. These sacrifices were meant to show the people in Old Testament times that Jesus would some day come to shed His Blood to redeem mankind from sin. The animals that were offered as sacrifices under the Law represented Jesus' coming to die for our sins, shedding His Blood on the Cross of Calvary to make atonement for the people. However, when Jesus died on Calvary, He had fulfilled all the sacrificial laws, and there was no more need for the Jews to keep them.


In [251]:
import requests
from bs4 import BeautifulSoup as bs

base_link = "https://apostolicfaithweca.org"

In [266]:
def get_lesson_link_junior(lessson_number):
    base_link_junior = f"https://apostolicfaithweca.org/sunday-school-lesson-library?llang=1&slc=2&title=&Slno={lessson_number}"
    r_j = requests.get(base_link_junior)
    soup_j = bs(r_j.content)
    junior_suffix = soup_j.find_all(class_ = "views-field views-field-title")[1].a['href']
    junior_title = soup_j.find_all(class_ = "views-field views-field-title")[1].a.text
    return junior_title, base_link+junior_suffix


In [267]:
def package_sources(source_names):
    source_names = source_names[:3]
    lesson_links = []
    for soure_name in source_names:
        lesson_number = soure_name.split("Lesson")[-1].split("_")[0]
        lesson_links.append(get_lesson_link_junior(lesson_number))

    return lesson_links
    

In [268]:
sources

['Lesson432_Junior.txt',
 'Lesson61_Junior.txt',
 'Lesson398_Junior.txt',
 'Lesson181_Junior.txt',
 'Lesson382_Junior.txt']

In [269]:
print(package_sources(sources))

[('JESUS, OUR REDEEMER', 'https://apostolicfaithweca.org/ssl/jesus-our-redeemer'), ('THE TEN COMMANDMENTS', 'https://apostolicfaithweca.org/node/3223'), ('THE LAW, A SCHOOLMASTER', 'https://apostolicfaithweca.org/ssl/law-schoolmaster-0')]
